In [1]:
# config
import os
PROJECT_ID = 'qwiklabs-gcp-00-08c0fbdbe48b'  # change here
BUCKET = PROJECT_ID + '_accelerator_demo'
REGION = 'us-central1'
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["BUCKET"] = BUCKET

!gcloud config set project {PROJECT_ID}
!gsutil mb gs://{BUCKET}
!gcloud config set compute/region {REGION}

Updated property [core/project].
Creating gs://qwiklabs-gcp-00-08c0fbdbe48b_accelerator_demo/...
Updated property [compute/region].


In [2]:
%%bash
mkdir train
echo "" > train/__init__.py
mkdir config

In [3]:
%%writefile train/model_definition.py
import tensorflow as tf

def create_model(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.BatchNormalization(input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10))
    model.add(tf.keras.layers.Activation('softmax'))
    return model

Writing train/model_definition.py


# Train on single Processor

In [4]:
%%writefile train/train_single_cpu_gpu.py
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

#Train model

model = model_definition.create_model(input_shape=x_train.shape[1:])
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
  loss='sparse_categorical_crossentropy',
  metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with single GPUs: {}".format(time.time() - start))

model.save_weights('./fashion_mnist_single.h5', overwrite=True)

Writing train/train_single_cpu_gpu.py


## Submit single worker with CPU

TODO 1: set module name defined above

In [5]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="single_cpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_single_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \

jobId: single_cpu_fashion_minst_20210528_061337
state: QUEUED


Job [single_cpu_fashion_minst_20210528_061337] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe single_cpu_fashion_minst_20210528_061337

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs single_cpu_fashion_minst_20210528_061337


## Submit single worker with single GPU

TODO 2: set module name defined above

TODO 3: set `scale-tier` to prefefined single GPU tier. Refer to [this page](https://cloud.google.com/ai-platform/training/docs/machine-types#scale_tiers)

In [6]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="single_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_single_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --scale-tier=BASIC_GPU

jobId: single_gpu_fashion_minst_20210528_061440
state: QUEUED


Job [single_gpu_fashion_minst_20210528_061440] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe single_gpu_fashion_minst_20210528_061440

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs single_gpu_fashion_minst_20210528_061440


# Train on multiple GPUs

TODO 4: Define distribute strategy for multiple GPU on single machine. Refer to [this page](https://www.tensorflow.org/guide/distributed_training)

TODO 5: Define scope of distribution strategy.

TODO 6: Specify 4 NVIDIA Tesla K80 GPU in config yaml file. Refere to [this page](https://cloud.google.com/ai-platform/training/docs/using-gpus)

TODO 7: Specify the module and config file in training submit command

In [7]:
%%writefile train/train_mult_cpu_gpu.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Multiple GPUs or CPUs ###################
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
###############################################################
    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with multiple GPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_mult_gpu.h5', overwrite=True)

Writing train/train_mult_cpu_gpu.py


In [8]:
%%writefile config/multi_gpu_config.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 4 T4 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80

Writing config/multi_gpu_config.yaml


In [9]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_cpu_gpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/multi_gpu_config.yaml

jobId: multi_gpu_fashion_minst_20210528_061655
state: QUEUED


Job [multi_gpu_fashion_minst_20210528_061655] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe multi_gpu_fashion_minst_20210528_061655

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs multi_gpu_fashion_minst_20210528_061655


# Example training on TPU

TODO 9: Write code for TPU training by Refering to [this page](https://www.tensorflow.org/guide/tpu). No need to pass any arguments in TPUClusterResolver class.

In [10]:
%%writefile train/train_tpu.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Run on TPUs ###################
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
###############################################################
    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with TPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_tpu.h5', overwrite=True)

Writing train/train_tpu.py


## Submit single worker with TPUv2 Pod (8 cores)

TODO 10: set module name defined above

TODO 11: set `scale-tier` to prefefined single TPU tier. Refer to [this page](https://cloud.google.com/ai-platform/training/docs/machine-types#scale_tiers)

In [11]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="tpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_tpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --scale-tier=BASIC_TPU \
  --region=us-central1

jobId: tpu_fashion_minst_20210528_061847
state: QUEUED


Job [tpu_fashion_minst_20210528_061847] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe tpu_fashion_minst_20210528_061847

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs tpu_fashion_minst_20210528_061847


## Submit single worker TPUv3 (8 cores)

TODO 12: specify 8 TPU V3 for accelerator by refering to [this page](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/AcceleratorType)

TODO 13: Specify the module and config file in training submit command

In [12]:
%%writefile config/tpuv3_config.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: n1-highcpu-16
  workerType: cloud_tpu
  workerCount: 1
  workerConfig:
    acceleratorConfig:
      type: TPU_V3
      count: 8

Writing config/tpuv3_config.yaml


In [13]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="tpu_v3_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_tpu \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/tpuv3_config.yaml

jobId: tpu_v3_fashion_minst_20210528_062012
state: QUEUED


Job [tpu_v3_fashion_minst_20210528_062012] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe tpu_v3_fashion_minst_20210528_062012

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs tpu_v3_fashion_minst_20210528_062012


# Train on multiple device with GPUs

TODO 14: Specify distribute strategy for multiple device GPU training

TODO 15: Specify master node with 4 NVIDIA TESLA K80, 3 worker node with 4 NVIDIA TESLA K80.

TODO 16: Specify the module and config file in training submit command

In [14]:
%%writefile train/train_mult_worker_mirrored.py
import os
import os
import time
import tensorflow as tf
import numpy as np
from . import model_definition

#Get data

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

##################### Run on multiple workers with GPU ###################
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

with strategy.scope():
###############################################################

    model = model_definition.create_model(input_shape=x_train.shape[1:])
    model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])
start = time.time()
model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)
print("Training time with multiple GPUs: {}".format(time.time() - start))
model.save_weights('./fashion_mnist_mult_mirrored.h5', overwrite=True)

Writing train/train_mult_worker_mirrored.py


In [15]:
%%writefile config/multi_worker_gpu.yaml
trainingInput:
  scaleTier: CUSTOM
  # Configure a master worker with 4 T4 GPUs
  masterType: n1-highcpu-16
  masterConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80
  # Configure 3 workers, each with 4 T4 GPUs
  workerCount: 3
  workerType: n1-highcpu-16
  workerConfig:
    acceleratorConfig:
      count: 4
      type: NVIDIA_TESLA_K80

Writing config/multi_worker_gpu.yaml


In [16]:
%%bash

now=$(date +"%Y%m%d_%H%M%S")
JOB_NAME="multi_worker_gpu_fashion_minst_$now"

gcloud ai-platform jobs submit training $JOB_NAME \
  --staging-bucket=gs://$BUCKET \
  --package-path=train \
  --module-name=train.train_mult_worker_mirrored \
  --runtime-version=2.1 \
  --python-version=3.7 \
  --region=us-central1 \
  --config config/multi_worker_gpu.yaml

jobId: multi_worker_gpu_fashion_minst_20210528_062146
state: QUEUED


Job [multi_worker_gpu_fashion_minst_20210528_062146] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe multi_worker_gpu_fashion_minst_20210528_062146

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs multi_worker_gpu_fashion_minst_20210528_062146


Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License